In [1]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Homework 4 — LLM-Powered Classification (Emotion dataset)\n",
    "\n",
    "**Notebook instructions:**\n",
    "- This notebook creates a small emotion dataset (20 rows), classifies each row using an LLM via OpenRouter, and saves `labeled.csv` plus `README.md`.\n",
    "- Do NOT store API keys in the repository. Set `OPENROUTER_API_KEY` as an environment variable.\n",
    "\n",
    "**How to run:**\n",
    "1. Install dependencies:\n",
    "   ```bash\n",
    "   pip install openai pandas tqdm matplotlib requests\n",
    "   ```\n",
    "2. Set API key in your environment (do not commit it):\n",
    "   ```bash\n",
    "   export OPENROUTER_API_KEY=\"<your_openrouter_key>\"\n",
    "   ```\n",
    "3. Run cells in order in a Jupyter environment."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1️ Create the Emotion Dataset"
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "import os\n",
    "import pandas as pd\n",
    "\n",
    "rows = [\n",
    "    \"I'm so happy I got accepted into the program!\",\n",
    "    \"I can't believe they lost my package again.\",\n",
    "    \"I really miss my hometown today.\",\n",
    "    \"That loud noise scared me so much.\",\n",
    "    \"The meeting went as usual, nothing special.\",\n",
    "    \"My friend surprised me with a birthday gift!\",\n",
    "    \"Why does everything have to go wrong today?!\",\n",
    "    \"I felt lonely the entire evening.\",\n",
    "    \"I hate how they never listen to feedback.\",\n",
    "    \"The thunderstorm last night was terrifying.\",\n",
    "    \"The new café near my house is amazing!\",\n",
    "    \"I'm frustrated with all these delays.\",\n",
    "    \"Hearing that song makes me emotional.\",\n",
    "    \"I'm worried about the exam next week.\",\n",
    "    \"The weather today is pretty normal.\",\n",
    "    \"I couldn't stop smiling all day.\",\n",
    "    \"They were yelling at us for no reason.\",\n",
    "    \"I cried after reading the news.\",\n",
    "    \"Walking alone late at night makes me nervous.\",\n",
    "    \"I don't really have an opinion about that.\"\n",
    "]\n",
    "\ndf = pd.DataFrame({\"text\": rows})\n",
    "OUT_DIR = \"assignments/homework 4/Ulyana Tsurkanu\"\n",
    "os.makedirs(OUT_DIR, exist_ok=True)\n",
    "DATA_CSV = os.path.join(OUT_DIR, \"data.csv\")\n",
    "df.to_csv(DATA_CSV, index=False, encoding=\"utf-8\")\n",
    "print(f\"Saved original data → {DATA_CSV}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2️ Label Set and Guide"
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "LABELS = [\"joy\", \"anger\", \"sadness\", \"fear\", \"neutral\"]\n",
    "LABEL_GUIDE = {\n",
    "    \"joy\": \"Expressions of happiness, delight, excitement, or positive surprise.\",\n",
    "    \"anger\": \"Expressions of irritation, annoyance, or strong displeasure.\",\n",
    "    \"sadness\": \"Expressions of sorrow, missing someone, crying, or feeling down.\",\n",
    "    \"fear\": \"Expressions of being afraid, scared, anxious or worried about danger.\",\n",
    "    \"neutral\": \"Statements that are factual, indifferent, or lack clear emotion.\"\n",
    "}\n",
    "\n",
    "# Save README.md\n",
    "README = f\"\"\"\n",
    "# Homework 4 Emotion Classification\n",
    "\n",
    "\n",
    "**Label set**:\n",
    "- joy: {LABEL_GUIDE['joy']}\n",
    "- anger: {LABEL_GUIDE['anger']}\n",
    "- sadness: {LABEL_GUIDE['sadness']}\n",
    "- fear: {LABEL_GUIDE['fear']}\n",
    "- neutral: {LABEL_GUIDE['neutral']}\n",
    "\n",
    "**Files created**:\n",
    "- data.csv    — original 20 rows with column 'text'\n",
    "- labeled.csv — same rows plus 'label' (predictions)\n",
    "\n",
    "**How to run**:\n",
    "1. Install dependencies:\n",
    "   pip install openai pandas tqdm matplotlib requests\n",
    "2. Set the OpenRouter API key in environment variable `OPENROUTER_API_KEY`.\n",
    "3. Execute the notebook cells. The notebook will call the LLM via OpenRouter for each row and save labeled.csv.\n",
    "\n",
    "**Important**: Do NOT include API keys in the repository. Use environment variables.\n",
    "\"\"\"\n",
    "\n",
    "with open(os.path.join(OUT_DIR, \"README.md\"), \"w\", encoding=\"utf-8\") as f:\n",
    "    f.write(README)\n",
    "print(\"Saved README.md\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3️ Prompt Design and Few-Shot Examples"
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "SYSTEM_PROMPT = (\n",
    "    \"You are a strict emotion classifier. Return ONLY valid JSON matching the schema:\\n\"\n",
    "    \"{ \\\"label\\\": \\\"joy|anger|sadness|fear|neutral\\\" }.\\n\"\n",
    "    \"Choose exactly one label from the allowed set.\\n\"\n",
    "    \"Do NOT add any explanation, extra text, or punctuation outside the JSON.\\n\"\n",
    ")\n",
    "\n",
    "FEW_SHOT = [\n",
    "    {\"text\": \"I got the job offer and I'm thrilled!\", \"label\": \"joy\"},\n",
    "    {\"text\": \"They broke my phone and I'm furious.\", \"label\": \"anger\"},\n",
    "    {\"text\": \"I miss my family so much right now.\", \"label\": \"sadness\"},\n",
    "    {\"text\": \"I'm terrified of walking home alone at night.\", \"label\": \"fear\"},\n",
    "    {\"text\": \"It is Wednesday and we have a meeting.\", \"label\": \"neutral\"}\n",
    "]\n",
    "\n",
    "EXAMPLE_PROMPT = \"Examples:\\n\"\n",
    "for ex in FEW_SHOT:\n",
    "    EXAMPLE_PROMPT += f\"Text: {ex['text']} => Label: {ex['label']}\\n\"\n",
    "\n",
    "FULL_SYSTEM_PROMPT = SYSTEM_PROMPT + \"\\n\" + EXAMPLE_PROMPT"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4️ OpenRouter Helper Function"
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "import os\n",
    "import requests\n",
    "import json\n",
    "\n",
    "MODEL = \"openrouter/auto\"\n",
    "OPENROUTER_API_KEY = os.environ.get(\"OPENROUTER_API_KEY\")\n",
    "if not OPENROUTER_API_KEY:\n",
    "    print(\"WARNING: OPENROUTER_API_KEY not set — API calls will not work.\")\n",
    "\n",
    "USE_OPENAI_CLIENT = False\n",
    "try:\n",
    "    from openai import OpenAI\n",
    "    USE_OPENAI_CLIENT = True\n",
    "except Exception:\n",
    "    try:\n",
    "        import openai\n",
    "        USE_OPENAI_CLIENT = True\n",
    "    except Exception:\n",
    "        USE_OPENAI_CLIENT = False\n",
    "\n",
    "def call_openrouter_chat(system_prompt, user_text, model=MODEL, api_key_env=\"OPENROUTER_API_KEY\"):\n",
    "    api_key = os.environ.get(api_key_env)\n",
    "    if not api_key:\n",
    "        return None, \"no_api_key\"\n",
    "\n",
    "    messages = [\n",
    "        {\"role\": \"system\", \"content\": system_prompt},\n",
    "        {\"role\": \"user\", \"content\": user_text}\n",
    "    ]\n",
    "\n",
    "    if USE_OPENAI_CLIENT:\n",
    "        try:\n",
    "            try:\n",
    "                client = OpenAI(api_key=api_key, base_url=\"https://openrouter.ai/api/v1\")\n",
    "            except TypeError:\n",
    "                client = OpenAI(api_key=api_key, api_base=\"https://openrouter.ai/api/v1\")\n",
    "            resp = client.chat.completions.create(model=model, messages=messages, temperature=0.0, max_tokens=30)\n",
    "            try:\n",
    "                content = resp.choices[0].message.content\n",
    "            except Exception:\n",
    "                content = resp['choices'][0]['message']['content']\n",
    "            return content, None\n",
    "        except Exception as e:\n",
    "            return None, f\"client_error: {str(e)}\"\n",
    "    else:\n",
    "        url = \"https://api.openrouter.ai/api/v1/chat/completions\"\n",
    "        headers = {\"Authorization\": f\"Bearer {api_key}\", \"Content-Type\": \"application/json\"}\n",
    "        payload = {\"model\": model, \"messages\": messages, \"temperature\": 0.0, \"max_tokens\": 30}\n",
    "        try:\n",
    "            r = requests.post(url, headers=headers, json=payload, timeout=30)\n",
    "        except Exception as e:\n",
    "            return None, f\"request_error: {str(e)}\"\n",
    "        if r.status_code != 200:\n",
    "            return None, f\"http_{r.status_code}: {r.text[:200]}\"\n",
    "        try:\n",
    "            j = r.json()\n",
    "            content = j['choices'][0]['message']['content']\n",
    "            return content, None\n",
    "        except Exception as e:\n",
    "            return None, f\"parse_error: {str(e)}\""
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5️ Loop over Dataset and Classify"
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "from tqdm import tqdm\n",
    "\n",
    "outputs = []\n",
    "errors = 0\n",
    "\n",
    "for idx, row in tqdm(df.iterrows(), total=len(df), desc=\"Classifying rows\"):\n",
    "    text = row['text']\n",
    "    resp_text, err = call_openrouter_chat(FULL_SYSTEM_PROMPT, text)\n",
    "    if err is not None:\n",
    "        outputs.append({\"text\": text, \"label\": \"error\", \"raw_response\": None, \"error\": err})\n",
    "        errors += 1\n",
    "        continue\n",
    "    if resp_text is None:\n",
    "        outputs.append({\"text\": text, \"label\": \"error\", \"raw_response\": None, \"error\": \"no_response\"})\n",
    "        errors += 1\n",
    "        continue\n",
    "\n",
    "    raw = resp_text.strip()\n",
    "    parsed_label = None\n",
    "    try:\n",
    "        first = raw.find('{')\n",
    "        last = raw.rfind('}')\n",
    "        candidate = raw[first:last+1] if (first != -1 and last != -1 and last > first) else raw\n",
    "        j = json.loads(candidate)\n",
    "        if isinstance(j, dict) and 'label' in j:\n",
    "            val = str(j['label']).strip().lower()\n",
    "            if val in LABELS:\n",
    "                parsed_label = val\n",
    "            else:\n",
    "                parsed_label = \"error\"\n",
    "        else:\n",
    "            parsed_label = \"error\"\n",
    "    except Exception:\n",
    "        parsed_label = \"error\"\n",
    "\n",
    "    if parsed_label == \"error\":\n",
    "        errors += 1\n",
    "    outputs.append({\"text\": text, \"label\": parsed_label, \"raw_response\": raw, \"error\": None if parsed_label != \"error\" else \"invalid_json_or_label\"})\n",
    "\n",
    "labeled_df = pd.DataFrame(outputs)\n",
    "LABELED_CSV = os.path.join(OUT_DIR, \"labeled.csv\")\n",
    "labeled_df.to_csv(LABELED_CSV, index=False, encoding=\"utf-8\")\n",
    "print(f\"Saved labeled results → {LABELED_CSV}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6️ Sanity Checks and Chart"
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "import matplotlib.pyplot as plt\n",
    "\n",
    "print('\\nLabel counts (including errors):')\n",
    "print(labeled_df['label'].value_counts(dropna=False))\n",
    "err_pct = (labeled_df['label'] == 'error').mean() * 100\n",
    "print(f\"\\nError percentage: {err_pct:.1f}% ({(labeled_df['label']=='error').sum()} rows)\")\n",
    "\n",
    "# Plot excluding 'error'\n",
    "plot_counts = labeled_df['label'].replace('error', pd.NA).value_counts()\n",
    "plt.figure(figsize=(6,4))\n",
    "plot_counts.plot(kind='bar')\n",
    "plt.title('Predicted label distribution (errors excluded)')\n",
    "plt.xlabel('label')\n",
    "plt.ylabel('count')\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 7️ Mini Analysis"
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "print('\\nMini analysis (5-8 sentences):')\n",
    "print('1) The model should reliably pick up explicit emotional cues: words like \"happy\", \"furious\", \"miss\", \"scared\" map well to labels.')\n",
    "print('2) Short, neutral-sounding lines (e.g., \"The meeting went as usual\") may be classified as neutral or occasionally misclassified.')\n",
    "print('3) Ambiguous phrases like \"hearing that song makes me emotional\" could be labeled joy or sadness depending on context; these are harder to separate.')\n",
    "print('4) Error responses happen when the model outputs non-JSON text or a label outside the allowed set; we mark these as \"error\" with no retries.')\n",
    "print('5) To improve reliability: include more few-shot examples, make the schema very explicit, shorten inputs to essential emotional cues, and keep temperature at 0.')\n",
    "\n",
    "from IPython.display import display\n",
    "print('\\nFirst 10 labeled rows:')\n",
    "display(labeled_df.head(10))"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "name": "python",
   "version": "3.11"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}


{'cells': [{'cell_type': 'markdown',
   'metadata': {},
   'source': ['# Homework 4 — LLM-Powered Classification (Emotion dataset)\n',
    '\n',
    '**Notebook instructions:**\n',
    '- This notebook creates a small emotion dataset (20 rows), classifies each row using an LLM via OpenRouter, and saves `labeled.csv` plus `README.md`.\n',
    '- Do NOT store API keys in the repository. Set `OPENROUTER_API_KEY` as an environment variable.\n',
    '\n',
    '**How to run:**\n',
    '1. Install dependencies:\n',
    '   ```bash\n',
    '   pip install openai pandas tqdm matplotlib requests\n',
    '   ```\n',
    '2. Set API key in your environment (do not commit it):\n',
    '   ```bash\n',
    '   export OPENROUTER_API_KEY="<your_openrouter_key>"\n',
    '   ```\n',
    '3. Run cells in order in a Jupyter environment.']},
  {'cell_type': 'markdown',
   'metadata': {},
   'source': ['## 1️ Create the Emotion Dataset']},
  {'cell_type': 'code',
   'metadata': {},
   'source': ['import os\n